In [1]:
import pandas as pd
import numpy as np

In [ ]:
dfa = pd.read_csv("/home/ruchika/Ruchika/trade/tr1.csv", chunksize=50)
df = pd.concat(dfa)
df = df.drop(['Unnamed: 0'], axis = 1)
df

In [ ]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='us')
df = df.sort_values(by='datetime',ascending=True, ignore_index=True)


In [ ]:
dfd = pd.DataFrame()
dfd['close'] = df.resample('1T', on='datetime').price.last()
dfd['open'] = df.resample('1T', on='datetime').price.first()
dfd['high'] = df.resample('1T', on='datetime').price.max()
dfd['low'] = df.resample('1T', on='datetime').price.min()
dfd['price'] = df.resample('1T', on='datetime').price.mean()
dfd['volume'] = df.resample('1T', on='datetime').amount.mean()
dfd['datetime'] = dfd.index

In [ ]:
dfb = pd.read_csv("/home/ruchika/Ruchika/quote/q1.csv",chunksize=50)
df1 = pd.concat(dfb)
df1 = df1.drop(['Unnamed: 0'], axis = 1)

In [ ]:
dfc = pd.read_csv("/home/ruchika/Ruchika/ticker/t1.csv",chunksize=50)
df2 = pd.concat(dfc)
df2 = df2.drop(['Unnamed: 0'], axis = 1)
df2['datetime'] = pd.to_datetime(df2['timestamp'], unit='us')
df2 = df2.sort_values(by='datetime',ascending=True, ignore_index=True)

In [ ]:
df1['datetime'] = pd.to_datetime(df1['timestamp'], unit='us')
df1 = df1.sort_values(by='datetime',ascending=True, ignore_index=True)
ask = df1.resample('1T', on='datetime').ask_price.mean()
bid = df1.resample('1T', on='datetime').bid_price.mean()
ab = ask - bid
dfd['delta'] = ab
dfd['open_interest'] = df2.resample('1T', on='datetime').open_interest.last()
#dfd.to_csv("/content/drive/MyDrive/btc_month.csv")

In [ ]:
dfd['datetime'] = dfd.index
dfd['date'] = pd.to_datetime(dfd['datetime']).dt.date
dfd = dfd.drop(['datetime'], axis=1)

In [ ]:
un = dfd['date'].unique()
dfd1 = pd.DataFrame()
dfd1['close'] = df.resample('D', on='datetime').price.last()
dfd1['open'] = df.resample('D', on='datetime').price.first()
dfd1['high'] = df.resample('D', on='datetime').price.max()
dfd1['low'] = df.resample('D', on='datetime').price.min()
dfd1['price'] = df.resample('D', on='datetime').price.mean()
dfd1['volume'] = df.resample('D', on='datetime').amount.mean()
dfd1['date'] = dfd1.index
high_low = dfd['high'] - dfd['low']
high_close = np.abs(dfd1['high'] - dfd1['close'].shift())
low_close = np.abs(dfd1['low'] - dfd1['close'].shift())
ranges = pd.concat([high_low, high_close, low_close], axis=1)
true_range = np.max(ranges, axis=1)
dfd['atr'] = true_range.rolling(14).sum()/14
dfd['vwap']=((dfd.price*dfd.volume)/dfd.volume)
dfd['bear_fractal'] = (dfd['high'].shift(4).lt(dfd['high'].shift(2)) & 
                      dfd['high'].shift(3).lt(dfd['high'].shift(2)) &
                      dfd['high'].shift(1).lt(dfd['high'].shift(2)) &
                      dfd['high'].lt(dfd['high'].shift(2)))
dfd['bull_fractal'] = (
                dfd['low'].shift(4).gt(dfd['low'].shift(2)) &
                dfd['low'].shift(3).gt(dfd['low'].shift(2)) &
                dfd['low'].shift(1).gt(dfd['low'].shift(2)) &
                dfd['low'].gt(dfd['high'].shift(2)))
dfd['symbol'] = "BTCUSD"
dd = dfd1.copy()
dfd.to_csv("/home/ruchika/Ruchika/data/1.csv")